# RL model voor 2x2x2 yard
In deze notebook wordt een RL model gemaakt voor een container yard van 2 bij 2 bij 2. Het gaat hier om een indeling waar 8 containers van 4 verschillende zeevaartschepen (die ze op komen halen) geplaatst moeten worden. Er zijn 2 containers per zeevaartschip, wat betekent dat de optimale oplossing simpel te bedenken en te checken is: Alle containers moeten dan in 1 keer gepakt kunnen worden, zoder verplaatsingen.

### Imports

In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import random
import pandas as pd
import numpy as np
import random
import os

from IPython.display import display
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack,DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

### Functie voor reward
Hierin worden penalty's gegeven voor slechte of niet toegestane zetten en rewards voor goede/optimale zetten. Per zet worden deze uitgedeeld, en het totaal levert de score van die 'game' op. Langzaam zou de reward steeds hoger moeten worden tijdens het trainen van het RL model.

In [2]:
# matrix[z_height][y_row][x_col]
def reward_functie_3D(matrix, breedte, plek, container, z_height, y_row, x_col):
    reward = 0
    
    #waar al een staat, -
    if plek !=0:
        reward -= 1
    
    #boven een lege, -
    if z_height > 0 and matrix[z_height-1][y_row][x_col] == 0:
        reward -= 1
    
    #niet twee dezelfde op begane grond, -
    if container in (matrix[z_height][0].tolist() or matrix[z_height][1].tolist()):
        reward -= 1
    
    #boven zelfde, +
    if z_height > 0 and matrix[z_height-1][y_row][x_col] == container:
        reward += 2
    
    return reward

### Environment maken
Er is hier gewerkt met als state/observation space een numpy array van 2 lijsten van 2 lijsten met 2 itemplaatsen. de eerste lijst van 2 lijsten moet de onderverdieping weergeven en de tweede de boven verdieping. links en rechts zijn de lange zijdes.

In [3]:
class Mijn_Env(Env):
    def __init__(self, breedte_matrix):
        self.x = breedte_matrix
        
        self.action_space = MultiDiscrete([self.x,self.x,self.x], dtype=int)
        #self.observation_space = Box(low=int(0),high=int(8),shape=(2,2,2), dtype=int)
        self.observation_space = Dict({'matrix':Box(low=int(0),high=int(8),shape=(2,2,2), dtype=int), 'container':Discrete(12)})
        self.state = np.array([[[0,0],[0,0]],[[0,0],[0,0]]])
        
        # nummers staan voor zeevaartschip waar containers voor bestemd zijn. Komen random volgorde binnen (want volgorde is onzeker)
        self.list_containers = list([1,1,2,2,3,3,4,4])
        random.shuffle(self.list_containers)
        
        # 8 te plaatsen containers
        self.duration = 8
        
    def step(self, action):
        self.duration -= 1
        container = self.list_containers.pop()
        y_row = action[0]    # 0 of 1
        x_col = action[1]    # 0 of 1
        z_height = action[2] # 0 of 1
        plek = self.state[z_height][y_row][x_col]
        
        # REWARD
        reward = reward_functie_3D(self.state, self.x, plek, container, z_height, y_row, x_col)
        
        # PLAATSEN
        self.state[z_height][y_row][x_col] = container
        
        # STOPPEN
        if self.duration <= 0:
            done=True
            obs = {'container':np.nan, 'matrix':self.state}
        else:
            done=False
            # geef mee aan model in obs: volgende te plaatsen container + huidige state van yard
            obs = {'container':self.list_containers[-1], 'matrix':self.state}
            
        info = {}
        
        return obs, reward, done, info
    
    def render(self, mode = 'human'):
        display(self.state)
    
    def reset(self):
        self.duration = 8
        self.state = np.array([[[0,0],[0,0]],[[0,0],[0,0]]])
        self.list_containers = list([1,1,2,2,3,3,4,4])
        random.shuffle(self.list_containers)

        obs = {'container':self.list_containers[-1], 'matrix':self.state}
        return obs
        #return self.state

In [4]:
env = Mijn_Env(2)

In [5]:
env.state

array([[[0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0]]])

In [6]:
print(env.list_containers)

[4, 1, 2, 1, 3, 2, 3, 4]


### Environment testen

In [7]:
episodes = 2
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

array([[[0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0]]])

array([[[0, 0],
        [0, 0]],

       [[0, 4],
        [0, 0]]])

array([[[0, 0],
        [0, 1]],

       [[0, 4],
        [0, 0]]])

array([[[0, 0],
        [0, 1]],

       [[0, 4],
        [0, 2]]])

array([[[0, 0],
        [3, 1]],

       [[0, 4],
        [0, 2]]])

...

array([[[4, 0],
        [0, 0]],

       [[0, 3],
        [2, 1]]])

array([[[4, 0],
        [0, 0]],

       [[0, 4],
        [2, 1]]])

array([[[3, 0],
        [0, 0]],

       [[0, 4],
        [2, 1]]])

array([[[3, 2],
        [0, 0]],

       [[0, 4],
        [2, 1]]])

Episode:2 Score:-8


### Environment trainen

In [8]:
model = PPO("MultiInputPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/opt/jupyterhub/anaconda/lib/python3.9/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484809662/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [9]:
time_steps=500000
model.learn(total_timesteps=int(time_steps))

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 8        |
|    ep_rew_mean     | -5.61    |
| time/              |          |
|    fps             | 1450     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 8           |
|    ep_rew_mean          | -5          |
| time/                   |             |
|    fps                  | 988         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014852754 |
|    clip_fraction        | 0.25        |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.07       |
|    explained_variance   | 0.0294      |
|    learning_rate        | 0.

### Model gebruiken/evalueren

In [10]:
# from stable_baselines3.common.monitor import Monitor
# env.reset()
# monitor = Monitor(env)
evaluate_policy(model, env, n_eval_episodes=1, return_episode_rewards = True, render=True)

/opt/jupyterhub/anaconda/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


array([[[0, 0],
        [0, 4]],

       [[0, 0],
        [0, 0]]])

array([[[0, 3],
        [0, 4]],

       [[0, 0],
        [0, 0]]])

array([[[0, 3],
        [0, 4]],

       [[0, 0],
        [0, 4]]])

array([[[0, 3],
        [0, 4]],

       [[0, 3],
        [0, 4]]])

array([[[1, 3],
        [0, 4]],

       [[0, 3],
        [0, 4]]])

array([[[1, 3],
        [2, 4]],

       [[0, 3],
        [0, 4]]])

array([[[1, 3],
        [2, 4]],

       [[0, 3],
        [2, 4]]])

array([[[0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0]]])

([8.0], [8])

# Conclusie
In het testvoorbeeld is te zien dat de optimale indeling gevonden is. Er staan in elke hoek 2 containers voor zelfde schip op elkaar, dus het maakt niet uit welk schip nu aankomt: de stacker kan bij de conatainers zonder dat die andere containers hoeft de verplaatsen. 